# 🚀 Quick Start - Panel Dashboard

**最新版本**: Phase 3.0 - Physical Isolation (v3.0)  
**特性**: 物理隔离 API (`data_values`) + 自动 HTML/PDF 导出

---

## 📋 本 Notebook 的步骤

1. **初始化环境** - 加载框架 + CSS 宽度修复
2. **加载数据** - 从 Parquet 文件加载
3. **生成基本 AI Context** - 数据结构信息
4. **生成完整 AI Prompt** - 包含所有文档（复制给 AI）
5. **选择维度** - 选择分析维度
6. **创建 Panel 仪表盘** - AI 生成分析逻辑
7. **导出 HTML** - 分享给他人

---


## 📦 Step 1: 初始化环境

加载框架 + CSS 宽度修复（让图表占满屏幕）

In [1]:
# ========================================
# 移除 JupyterLab 宽度限制（关键！）
# ========================================
from IPython.display import HTML, display

display(HTML("""
<style>
    /* 核心：解除 JupyterLab 4 的宽度限制 */
    .jp-Notebook { 
        --jp-notebook-max-width: 100% !important; 
    }
    
    /* 确保所有输出容器占满宽度 */
    .jp-Notebook-cell, 
    .jp-Cell-outputWrapper, 
    .jp-OutputArea-output, 
    .jp-OutputArea-child { 
        max-width: none !important; 
        width: 100% !important; 
    }
    
    /* Panel 根容器强制铺满 */
    .bk-root, .bk-root > .bk { 
        width: 100% !important; 
        max-width: none !important; 
    }
</style>
"""))

print("✅ JupyterLab 宽度限制已移除")

# ========================================
# 加载框架
# ========================================
import polars as pl
import plotly.express as px
import panel as pn

from src.session import DataSession
from src.dashboard import PanelDashboardBuilder
from src.utils import print_markdown_table

# 初始化 Panel（全局设置 stretch_width）
pn.extension('plotly', sizing_mode='stretch_width')

print("✅ 环境初始化完成")
print("📚 框架版本: Phase 3.0 - Physical Isolation (v3.0)")


✅ JupyterLab 宽度限制已移除


✅ 环境初始化完成
📚 框架版本: Phase 3.0 - Physical Isolation (v3.0)


## 📂 Step 2: 加载数据

从 Parquet 文件加载数据

In [2]:
# 创建数据会话
session = DataSession()

# 加载数据（替换为你的数据文件）
session.load("insurance_data_cleaned", alias="df")  # 或 session.load("your_data.parquet", alias="df")

# 验证数据
print(f"✅ 数据已加载: {df_df.height:,} 行 × {df_df.width} 列")
print(f"📊 数据预览:\n")
print_markdown_table(df_df.head(5))


✅ 已加载: df_df (insurance_data_cleaned)
✅ 数据已加载: 776,739 行 × 69 列
📊 数据预览:



|   业务年度 |   机构代码 | 机构名称                         |   业务险种 |                 保单号 |   危险单位号 | 支付币别   | 业务来源   | 是否联共保   |   联共保我司占比 |         地址代码 | 标的地址或船名                                                   | 被保险人名称               | 保险起期   | 保险止期   |   风险代码 | 风险代码描述                                         |   风险等级 |      总保额 |   总保费 |   总保费_原币 |   手续费 |   总已决赔款 |   总未决赔款 |    自留保额 |   自留保费 |   自留已决 |   自留未决 |   附加自留保额 |   附加自留保费 |   附加自留已决 |   附加自留未决 |   协议分保额 |   协议分保费 |   协议摊回费用 |   协议已决 |   协议未决 |   成数分保额 |   成数分保费 |   成数摊回费用 |   成数已决 |   成数未决 |   溢额分保额 |   溢额分保费 |   溢额摊回费用 |   溢额已决 |   溢额未决 |   OPENCOVER分保额 |   OPENCOVER分保费 |   OPENCOVER摊回费用 |   OPENCOVER已决 |   OPENCOVER未决 |   临分分保额 |   临分分保费 |   临分摊回费用 |   临分已决 |   临分未决 | 占用性质名称   | 行业类别名称   | 境内境外   |   分入手续费率 |   分入手续费 |   分入出单费比例 |   分入出单费 |   分入经纪费率 |   分入经纪费 | 分入经纪人名称   | 直保代理人名称                           | 归属机构   |
|-----------:|-----------:|:---------------------------------|-----------:|-----------------------:|-------------:|:-----------|:-----------|:-------------|-----------------:|-----------------:|:-----------------------------------------------------------------|:---------------------------|:-----------|:-----------|-----------:|:-----------------------------------------------------|-----------:|------------:|---------:|--------------:|---------:|-------------:|-------------:|------------:|-----------:|-----------:|-----------:|---------------:|---------------:|---------------:|---------------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|-------------:|-------------:|---------------:|-----------:|-----------:|------------------:|------------------:|--------------------:|----------------:|----------------:|-------------:|-------------:|---------------:|-----------:|-----------:|:---------------|:---------------|:-----------|---------------:|-------------:|-----------------:|-------------:|---------------:|-------------:|:-----------------|:-----------------------------------------|:-----------|
|       2024 |   44030033 | 深圳分公司工商保险业务部         |       0102 | 3010220244403000000577 |            1 | CNY        | 直接业务   | 非联共保     |            100   | 4403003300008323 | 广东省-深圳市-福田区滨河大道与益田路交界东南皇庭世纪花园1号楼14A | 崔鹏                       | 2024-04-23 | 2027-07-22 |      24107 | 中行渠道财产险类一般行业                             |          1 | 7.8329e+06  |  1921.27 |       1921.27 |     0    |            0 |            0 | 7.8329e+06  |    1921.27 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 综合商业类     |                | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  |                                          | 深圳       |
|       2016 |   37070004 | 山东分公司潍坊中心支公司银保团队 |       0101 | 3010120163707000000006 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3707000300000714 | 山东省潍坊市滨海经济开发区新海大街99号                           | 潍坊友容实业有限公司       | 2016-01-06 | 2017-01-05 |      11160 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 3.97982e+07 | 27858.8  |      27858.8  |  4178.81 |            0 |            0 | 3.97982e+07 |   27858.8  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 建筑安装业     | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司潍坊分行             | 山东       |
|       2023 |   31030001 | 上海自贸试验区分公司经理室       |       0104 | 3010420233103000001350 |            1 | CNY        | 经纪业务   | 从共         |             10.5 | 3100001700042357 | 新疆维吾尔自治区哈密地区哈密市伊州区                             | 中国广核集团               | 2023-02-01 | 2023-09-30 |      23056 | 市场渠道共保财产险类电厂及水电站（不含联合循环电厂） |          2 | 1.33116e+08 | 21421.9  |      21421.9  |  3406.08 |            0 |            0 | 1.2e+08     |   19311.2  |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  1.31159e+07 |      2110.69 |         675.42 |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 二级工业       | 其他电力生产   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中广核保险经纪有限责任公司               | 上海       |
|       2020 |   33070007 | 浙江分公司金华中心支公司银保团队 |       0101 | 3010120203307000000114 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3307000800003824 | 浙江省金华市兰溪市兰江街道毕家村                                 | 兰溪市三江仓储发展有限公司 | 2020-05-23 | 2021-05-22 |      20107 | 中行渠道财产险类一般行业                             |          3 | 1.23e+06    |  1160.38 |       1160.38 |     0    |            0 |            0 | 1.23e+06    |    1160.38 |          0 |          0 |              0 |              0 |              0 |              0 |     0        |            0 |            0   |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 工业           | 内河货物运输   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行股份有限公司兰溪支行             | 浙江       |
|       2011 |   32010004 | 江苏分公司南京营业部银保团队     |       0102 |  301022010320100000012 |            1 | CNY        | 代理业务   | 非联共保     |            100   | 3200001000000077 | 江苏省南京市太平北路106号第四至七层                              | 宏图三胞高科技术有限公司   | 2011-01-01 | 2011-12-31 |      10383 | 中行渠道非共保或主共保财产险类一般行业               |          2 | 2.95e+07    | 14750    |      14750    |  2360    |            0 |            0 | 2.832e+07   |   14160    |          0 |          0 |              0 |              0 |              0 |              0 |     1.18e+06 |          590 |          241.9 |          0 |          0 |            0 |            0 |              0 |          0 |          0 |  0           |         0    |           0    |          0 |          0 |                 0 |                 0 |                   0 |               0 |               0 |            0 |            0 |              0 |          0 |          0 | 商业           | 其他科技服务   | 境内业务   |            nan |          nan |              nan |          nan |            nan |          nan |                  | 中国银行南京地区鼓楼支行公司板块公司业务 | 江苏       |

## 🤖 Step 3: 生成基本 AI Context

生成数据概览

In [3]:
# 生成基本 AI Context
print("📋 数据结构信息:\n")
print("=" * 80)
print(session.get_ai_context())
print("=" * 80)

print("\n💡 提示: 下一步会生成完整的 AI Prompt")


📋 数据结构信息:

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float64)
- `溢额未决` (Float64

## 📋 Step 4: 生成完整 AI Prompt

**运行这个 cell，然后复制全部输出给 AI**

这个 cell 会输出：
- 数据结构信息
- 完整的 Panel Dashboard 使用指南
- 代码模板
- 常见错误对比
- 检查清单


In [4]:
# ========================================
# 📋 给 AI 的完整信息（复制全部输出给 AI）
# ========================================

# 直接执行最新的 Step 4 模板
import os

# 自动找到项目根目录
def find_project_root():
    """向上查找包含 src/ 和 notebooks/templates/ 的项目根目录"""
    current = os.path.abspath('.')
    while current != '/':
        # 检查是否存在项目标志
        if (os.path.exists(os.path.join(current, 'src')) and 
            os.path.exists(os.path.join(current, 'notebooks', 'templates'))):
            return current
        # 向上一级
        current = os.path.dirname(current)
    # 找不到就返回当前目录
    return os.path.abspath('.')

project_root = find_project_root()
step4_path = os.path.join(project_root, 'notebooks', 'templates', 'step4_standalone.py')

# 调试信息（可选，帮助诊断）
# print(f"项目根目录: {project_root}")
# print(f"Step 4 路径: {step4_path}")
# print(f"文件存在: {os.path.exists(step4_path)}")

if os.path.exists(step4_path):
    exec(open(step4_path).read())
else:
    # 如果文件找不到，使用嵌入版本
    print("=" * 80)
    print("📋 **复制以下所有内容给 AI**")
    print("=" * 80)
    print()
    print("## 📊 数据结构")
    print()
    print(session.get_ai_context())
    print()
    print("=" * 80)
    print("## 📚 Panel Dashboard 完整使用指南")
    print("=" * 80)
    print()
    print("⚠️ 规则 0: 禁止硬编码任何维度！")
    print("物理隔离规范 (v3.0):")
    print("- 必须使用: filters = dashboard.data_values")
    print("- 必须使用: agg_dim = dashboard.widgets['_aggregation_dimension'].value")
    print("物理隔离规范 (v3.0):")
    print("- 必须使用: filters = dashboard.data_values")
    print("- 必须使用: agg_dim = dashboard.widgets['_aggregation_dimension'].value")
    
    
    print()
    print("详细说明见: notebooks/templates/step4_standalone.py")
    print("=" * 80)


📋 **复制以下所有内容给 AI**

## 📊 数据结构

# 📊 当前数据会话

已加载的数据集：

## `df_df` (insurance_data_cleaned)
**数据量：** 776,739 行 × 69 列

**字段：**
- `业务年度` (String)
- `机构代码` (String)
- `机构名称` (String)
- `业务险种` (String)
- `保单号` (String)
- `危险单位号` (String)
- `支付币别` (String)
- `业务来源` (String)
- `是否联共保` (String)
- `联共保我司占比` (Float64)
- `地址代码` (String)
- `标的地址或船名` (String)
- `被保险人名称` (String)
- `保险起期` (String)
- `保险止期` (String)
- `风险代码` (String)
- `风险代码描述` (String)
- `风险等级` (String)
- `总保额` (Float64)
- `总保费` (Float64)
- `总保费_原币` (Float64)
- `手续费` (Float64)
- `总已决赔款` (Float64)
- `总未决赔款` (Float64)
- `自留保额` (Float64)
- `自留保费` (Float64)
- `自留已决` (Float64)
- `自留未决` (Float64)
- `附加自留保额` (Float64)
- `附加自留保费` (Float64)
- `附加自留已决` (Float64)
- `附加自留未决` (Float64)
- `协议分保额` (Float64)
- `协议分保费` (Float64)
- `协议摊回费用` (Float64)
- `协议已决` (Float64)
- `协议未决` (Float64)
- `成数分保额` (Int64)
- `成数分保费` (Int64)
- `成数摊回费用` (Int64)
- `成数已决` (Int64)
- `成数未决` (Int64)
- `溢额分保额` (Float64)
- `溢额分保费` (Float64)
- `溢额摊回费用` (Float64)
- `溢额已决` (Float6

## 🎯 Step 5: 选择分析维度

选择你想要分析的维度字段

In [5]:
# 👤 选择你要分析的维度
# 修改下面的列表，添加或删除维度

selected_dimensions = [
    '业务年度',
    '业务险种',
    '归属机构'  # 取消注释以启用
    # '境内境外',  # 取消注释以启用
]

print(f"✅ 已选择 {len(selected_dimensions)} 个维度:")
for dim in selected_dimensions:
    if dim in df_df.columns:
        n_unique = df_df[dim].n_unique()
        print(f"  - {dim} ({n_unique:,} 个唯一值)")
    else:
        print(f"  ⚠️  {dim} - 不存在于数据中")

print("\n💡 下一步: 创建 Panel 仪表盘")


✅ 已选择 3 个维度:
  - 业务年度 (25 个唯一值)
  - 业务险种 (26 个唯一值)
  - 归属机构 (30 个唯一值)

💡 下一步: 创建 Panel 仪表盘


## 📊 Step 6: 创建 Panel 仪表盘

自动创建仪表盘控件，然后让 AI 生成分析逻辑

In [7]:
# 创建 Panel 仪表盘（自动生成控件）
dashboard = PanelDashboardBuilder.from_data(
    df_df,
    dimensions=selected_dimensions,
    title="📊 数据分析仪表盘"
)

print("\n💡 下一步: 让 AI 生成分析逻辑")
print("\n📝 给 AI 的提示:")
print("="  * 80)
print("请严格遵守 main.md 中的 v3.0 规范。")
print("核心原则：严禁遍历 .widgets 进行数据过滤，必须遍历 .data_values 以实现业务列与功能列的物理隔离。")
print(f"已选择维度: {selected_dimensions}")
print("需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]")
print("="  * 80)


🎨 从数据创建仪表盘: 📊 数据分析仪表盘
📊 数据维度: 776,739 行 × 69 列
🔧 配置维度字段: 业务年度, 业务险种, 归属机构



  ✅ 业务年度: MultiChoice (25 个选项 + 全选, 默认选 1 个)
  ✅ 业务险种: MultiChoice (26 个选项 + 全选, 默认选 1 个)
  ✅ 归属机构: MultiChoice (30 个选项 + 全选, 默认选 1 个)

  ✅ 聚合维度选择器: Select (3 个维度可选, 默认: 业务年度)

✅ 仪表盘控件创建完成 (4 个控件)
💡 下一步: 使用 dashboard.set_update_function(your_function)

📄 等效代码（可复制用于自定义）:

```python
import panel as pn
import polars as pl

dashboard = PanelDashboardBuilder(title="📊 数据分析仪表盘")

# 示例: 业务年度
widget = pn.widgets.Select(name='业务年度', options=[...])
dashboard.widgets['业务年度'] = widget

# 示例: 业务险种
widget = pn.widgets.Select(name='业务险种', options=[...])
dashboard.widgets['业务险种'] = widget

# 定义更新函数
@pn.depends(*dashboard.widgets.values())
def update(*args):
    # 你的分析逻辑
    return fig

dashboard.set_update_function(update)
dashboard.show()  # Jupyter 显示
dashboard.save('output.html')  # 导出 HTML
```



💡 下一步: 让 AI 生成分析逻辑

📝 给 AI 的提示:
请严格遵守 main.md 中的 v3.0 规范。
核心原则：严禁遍历 .widgets 进行数据过滤，必须遍历 .data_values 以实现业务列与功能列的物理隔离。
已选择维度: ['业务年度', '业务险种', '归属机构']
需求: [描述你的分析需求，例如: 各险种保费排名 Top 10]


## 🤖 Step 7: AI 生成分析逻辑

**在这里粘贴 AI 生成的代码**

AI 应该生成：
1. `@pn.depends` 装饰的更新函数
2. 数据过滤和聚合逻辑
3. Plotly 图表
4. `dashboard.show()` 和 `dashboard.save()`


In [8]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import polars as pl
import panel as pn
from src.utils import print_markdown_table

# ⚠️ 假设 dashboard 对象已经存在（由你之前定义）

@pn.depends(*dashboard.widgets.values())
def update_dashboard(*args):
    # 1. 🛡️ 物理隔离：仅获取数据维度的筛选值
    # - 使用 .data_values 避免系统控件干扰
    filters = dashboard.data_values
    
    # 2. ⚡️ 动态聚合：获取当前选择的分组维度
    # - 动态获取 X 轴，不硬编码
    agg_dim = dashboard.widgets['_aggregation_dimension'].value
    
    # --- A. 数据过滤 ---
    filtered = df_df
    for col, val in filters.items():
        if isinstance(val, list):
            if '全选' not in val:
                filtered = filtered.filter(pl.col(col).is_in(val))
        elif val != '全选':
            filtered = filtered.filter(pl.col(col) == val)
            
    # --- B. 聚合计算 ---
    # 计算保费（规模）和 手续费（成本）
    # - 使用 pl.len() 替代 pl.count()
    analysis = filtered.group_by(agg_dim).agg([
        pl.col('总保费').sum().alias('保费'),
        pl.col('手续费').sum().alias('获取成本'),
        pl.len().alias('保单数')
    ])
    
    # 计算成本率 (处理分母为0的情况)
    analysis = analysis.with_columns(
        (pl.col('获取成本') / pl.col('保费')).fill_nan(0).alias('成本率')
    )
    
    # --- C. 智能排序 (针对"最近十年"需求) ---
    if '年度' in agg_dim or 'Year' in agg_dim:
        # 如果是时间维度，按时间正序排列
        analysis = analysis.sort(agg_dim)
    else:
        # 其他维度（如机构、险种），按保费规模降序，关注头部
        analysis = analysis.sort('保费', descending=True)
    
    # 转换为 Pandas 用于绘图 (取前 15 条以防维度过多)
    chart_data = analysis.head(15).to_pandas()

    # --- D. 可视化 (双轴图表: 保费 vs 成本率) ---
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # 柱状图：保费规模
    fig.add_trace(
        go.Bar(
            x=chart_data[agg_dim], 
            y=chart_data['保费'], 
            name="总保费 (左轴)",
            marker_color='#636EFA',
            opacity=0.8
        ),
        secondary_y=False
    )

    # 折线图：成本率趋势
    fig.add_trace(
        go.Scatter(
            x=chart_data[agg_dim], 
            y=chart_data['成本率'], 
            name="成本率 (右轴)",
            mode='lines+markers+text',
            line=dict(color='#EF553B', width=3),
            text=[f"{v:.1%}" for v in chart_data['成本率']],
            textposition="top center",
            textfont=dict(size=10)
        ),
        secondary_y=True
    )

    # 布局设置 (自适应宽度)
    # - 设置 autosize=True 且移除宽度限制
    fig.update_layout(
        title=f'📊 保费规模 vs 获取成本率 ({agg_dim})',
        height=600,
        autosize=True,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=40, r=40, t=80, b=40),
        hovermode="x unified"
    )
    
    # 坐标轴设置
    fig.update_yaxes(title_text="保费 (元)", secondary_y=False)
    fig.update_yaxes(title_text="成本率 (%)", tickformat='.1%', secondary_y=True, 
                     range=[0, max(chart_data['成本率'].max(), 0.1) * 1.2]) # 动态调整Y轴范围

    # --- E. Markdown 报告输出 ---
    print(f"## 📈 分析报告：{agg_dim}维度\n")
    
    total_prem = analysis['保费'].sum()
    avg_cost_ratio = (analysis['获取成本'].sum() / total_prem) if total_prem > 0 else 0
    
    print(f"- **总保费规模**: {total_prem:,.2f} 元")
    print(f"- **综合获取成本率**: {avg_cost_ratio:.2%}")
    print(f"- **数据覆盖**: {filtered.height:,} 条记录\n")
    
    print("### 详细数据摘要")
    # 格式化表格用于展示
    display_tbl = analysis.select([
        pl.col(agg_dim),
        pl.col('保费').map_elements(lambda x: f"{x:,.0f}", return_dtype=pl.String),
        pl.col('获取成本').map_elements(lambda x: f"{x:,.0f}", return_dtype=pl.String),
        (pl.col('成本率') * 100).round(2).cast(pl.String) + '%'
    ]).head(10)
    
    print_markdown_table(display_tbl)
    
    return fig

# 3. 绑定并显示
dashboard.set_update_function(update_dashboard)
dashboard.show()

## 📈 分析报告：业务年度维度

- **总保费规模**: 13,831,756,844.14 元
- **综合获取成本率**: 15.15%
- **数据覆盖**: 776,739 条记录

### 详细数据摘要


|   业务年度 | 保费        | 获取成本   | 成本率   |
|-----------:|:------------|:-----------|:---------|
|       2001 | 86,790      | 0          | 0.0%     |
|       2002 | 11,854,107  | 362,001    | 3.05%    |
|       2003 | 33,687,661  | 3,038,561  | 9.02%    |
|       2004 | 48,785,653  | 1,595,579  | 3.27%    |
|       2005 | 49,690,640  | 2,549,978  | 5.13%    |
|       2006 | 96,746,301  | 6,823,424  | 7.05%    |
|       2007 | 140,237,598 | 13,731,297 | 9.79%    |
|       2008 | 267,649,751 | 34,170,309 | 12.77%   |
|       2009 | 381,493,256 | 54,449,774 | 14.27%   |
|       2010 | 522,582,721 | 74,339,943 | 14.23%   |

Column(sizing_mode='stretch_width')
    [0] Markdown(str, sizing_mode='stretch_width')
    [1] FlexBox(objects=[MultiChoice(name='📊 业务年度'...], sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

## 📥 Step 8: 导出 PDF 报告

此步骤会自动捕获当前仪表盘的所有图表和过滤配置，生成一份专业美观的 PDF 报告。报告将保存在 `outputs/reports/` 目录下。

In [9]:
from src.exporter import ReportExporter
import os
from IPython.display import FileLink, display

# 1. 自动检测环境中的 dashboard 对象
target_obj = globals().get('dashboard')

if target_obj:
    # 2. 执行导出
    # 你可以修改报告标题和文件名
    report_path = ReportExporter.export_to_pdf(
        dashboard=target_obj,
        title="数据分析深度报告",
        author="Harold AI 分析助手",
        filename="analysis_report"
    )
    
    # 3. 显示结果
    if report_path and os.path.exists(report_path):
        print("\n✅ 报告生成成功！")
        display(FileLink(report_path))
else:
    print("❌ 错误: 未找到 'dashboard' 对象，请确保 Step 6 & 7 已执行并正常显示。")

🚀 开始生成 PDF 报告: analysis_report.pdf...
🔍 正在提取图表...
  🔄 尝试调用 update_function 生成图表...
## 📈 分析报告：业务年度维度

- **总保费规模**: 13,831,756,844.14 元
- **综合获取成本率**: 15.15%
- **数据覆盖**: 776,739 条记录

### 详细数据摘要


|   业务年度 | 保费        | 获取成本   | 成本率   |
|-----------:|:------------|:-----------|:---------|
|       2001 | 86,790      | 0          | 0.0%     |
|       2002 | 11,854,107  | 362,001    | 3.05%    |
|       2003 | 33,687,661  | 3,038,561  | 9.02%    |
|       2004 | 48,785,653  | 1,595,579  | 3.27%    |
|       2005 | 49,690,640  | 2,549,978  | 5.13%    |
|       2006 | 96,746,301  | 6,823,424  | 7.05%    |
|       2007 | 140,237,598 | 13,731,297 | 9.79%    |
|       2008 | 267,649,751 | 34,170,309 | 12.77%   |
|       2009 | 381,493,256 | 54,449,774 | 14.27%   |
|       2010 | 522,582,721 | 74,339,943 | 14.23%   |

  ✅ 通过调用 update_function 获取到 1 个图表
⚠️ 尝试加载 /System/Library/Fonts/Hiragino Sans GB.ttc 失败: TTC file "/System/Library/Fonts/Hiragino Sans GB.ttc": postscript outlines are not supported
✅ 成功加载中文字体: /System/Library/Fonts/STHeiti Medium.ttc

📊 开始处理 1 个图表...
  🖼️ 正在导出图表 1/1: 📊 保费规模 vs 获取成本率 (业务年度)...
    ✅ 成功生成图片 (249.5 KB)
✅ 报告已成功导出至: outputs/reports/analysis_report.pdf

✅ 报告生成成功！


/Users/harold/working/Jupyter_AI_DataAnalyze/notebooks/02_analysis/outputs/reports/analysis_report.pdf